In [ ]:
import dotenv
import os
os.environ.setdefault("uri", "config.settings.local")
os.environ.setdefault("user", "config.settings.local")
os.environ.setdefault("password", "config.settings.local")



In [ ]:
import requests
import json
session=requests.Session()
login_payload={
    "email":"mukkawarshravan04@gmail.com",
    "password":"2250213"
}
login_res=session.post("http://localhost:5000/api/v1/users/signIn",json=login_payload)
cookies=session.cookies.get_dict()

groups=session.get("http://localhost:5000/api/v1/group/getG")
# print(groups.json())
print(json.dumps(groups.json()))

{'statusCode': 200, 'data': [{'_id': '68566e3e594730eadfd4c0c1', 'name': '407', 'leader': {'_id': '68566cb3594730eadfd4bdde', 'fullName': 'John'}, 'members': ['68566cb3594730eadfd4bdde', '68566cafbe81f4c113ca432c'], 'resources': [], 'description': 'hostel', 'whiteboard': '68566e3e594730eadfd4c0bf', 'createdAt': '2025-06-21T08:33:02.126Z', '__v': 0}, {'_id': '6857f71ca29cb235823a2ab9', 'name': 'Shravan mukkawar', 'leader': {'_id': '68566cafbe81f4c113ca432c', 'fullName': 'Shravan Mukkawar'}, 'members': ['68566cafbe81f4c113ca432c'], 'resources': [], 'description': 'Main group', 'tags': 'React', 'category': 'Web Development', 'whiteboard': '6857f71ca29cb235823a2ab7', 'createdAt': '2025-06-22T12:29:16.860Z', '__v': 0}, {'_id': '68580dee735f02eabc6c4a65', 'name': 'Prompt Engineers', 'leader': {'_id': '68566cafbe81f4c113ca432c', 'fullName': 'Shravan Mukkawar'}, 'members': ['68566cafbe81f4c113ca432c'], 'resources': [], 'description': 'Work on LangChain, LLMs and smart agents', 'tags': 'LangCha

In [ ]:
import json

# Get JSON from response
data = groups.json()

# Get the list of groups
groups_list = data.get("data", [])

# Open a text file to write the output
with open("groups_info.txt", "w", encoding="utf-8") as f:
    for i, group in enumerate(groups_list, start=1):
        name = group.get("name", "N/A")
        description = group.get("description", "N/A")
        category = group.get("category", "N/A")
        tags = group.get("tags", "N/A")

        f.write(f"Group {i}:\n")
        f.write(f"Name       : {name}\n")
        f.write(f"Description: {description}\n")
        f.write(f"Category   : {category}\n")
        f.write(f"Tags       : {tags}\n")
        f.write("-" * 40 + "\n")

print("Group info saved to groups_info.txt")


Group info saved to groups_info.txt


In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

llm=Ollama(model='llama2:7b')


C:\Users\mukka\AppData\Local\Temp\ipykernel_22480\1620002507.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model='llama2:7b')


In [2]:
groups = []

with open("groups_info.txt", "r", encoding="utf-8") as f:
    content = f.read().split("----------------------------------------\n")
    for block in content:
        if block.strip():
            groups.append(block.strip())

print(groups)

['Group 1:\nName       : 407\nDescription: hostel\nCategory   : N/A\nTags       : N/A', 'Group 2:\nName       : Shravan mukkawar\nDescription: Main group\nCategory   : Web Development\nTags       : React', 'Group 3:\nName       : Prompt Engineers\nDescription: Work on LangChain, LLMs and smart agents\nCategory   : AI & ML\nTags       : LangChain,LLM,Agents', 'Group 4:\nName       : React Masters\nDescription: Deep dive into React, hooks, context, and advanced patterns.\nCategory   : Web Development\nTags       : React, JavaScript, Frontend', 'Group 5:\nName       : LangChain Coders\nDescription: Discuss advanced LangChain projects and agents\nCategory   : AI & ML\nTags       : LangChain,LLMs,Embeddings', 'Group 6:\nName       : Arduino Hackers\nDescription: Tinker with sensors and microcontrollers.\nCategory   : Embedded Systems\nTags       : Arduino, IoT, Electronics', 'Group 7:\nName       : Generative AI Creators Hub\nDescription: A community for exploring, experimenting with, and b

In [14]:
from langchain.schema import Document

documents = []

for group in groups:
    lines = group.splitlines()
    metadata = {}

    for line in lines:
        if line.startswith("Name"):
            metadata["name"] = line.split(":", 1)[1].strip()
        elif line.startswith("Description"):
            metadata["description"] = line.split(":", 1)[1].strip()
        elif line.startswith("Category"):
            metadata["category"] = line.split(":", 1)[1].strip()
        elif line.startswith("Tags"):
            tags_str = line.split(":", 1)[1].strip()
            
            metadata["tags"] = [tag.strip() for tag in tags_str.split(",")]

    documents.append(Document(page_content=group, metadata=metadata))

# Optional: Print one to verify
print(documents)


[Document(metadata={'name': '407', 'description': 'hostel', 'category': 'N/A', 'tags': ['N/A']}, page_content='Group 1:\nName       : 407\nDescription: hostel\nCategory   : N/A\nTags       : N/A'), Document(metadata={'name': 'Shravan mukkawar', 'description': 'Main group', 'category': 'Web Development', 'tags': ['React']}, page_content='Group 2:\nName       : Shravan mukkawar\nDescription: Main group\nCategory   : Web Development\nTags       : React'), Document(metadata={'name': 'Prompt Engineers', 'description': 'Work on LangChain, LLMs and smart agents', 'category': 'AI & ML', 'tags': ['LangChain', 'LLM', 'Agents']}, page_content='Group 3:\nName       : Prompt Engineers\nDescription: Work on LangChain, LLMs and smart agents\nCategory   : AI & ML\nTags       : LangChain,LLM,Agents'), Document(metadata={'name': 'React Masters', 'description': 'Deep dive into React, hooks, context, and advanced patterns.', 'category': 'Web Development', 'tags': ['React', 'JavaScript', 'Frontend']}, page

In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Milvus
from langchain.schema import Document

embedding_model = OllamaEmbeddings(model='llama2:7b')  

# Zilliz Cloud connection settings
connection_args = {
    "uri": "https://in03-26dd51b46ed6273.serverless.gcp-us-west1.cloud.zilliz.com",
    "user": "db_26dd51b46ed6273",
    "password": "Tm1>C5CkrNL}BOcD",
    "secure": True
}

# Create or connect to Milvus (Zilliz Cloud) vector store
vectorstore = Milvus.from_documents(
    documents,
    embedding_model,
    connection_args=connection_args,
    collection_name="zilliz_llama_test"
)


Failure to create collection, unrecognized dtype for key: tags


ValueError: Unrecognized datatype for tags.

In [ ]:
vectorstore.save_local("./Milvus")
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
print(retriever.get_relevant_documents("Blockchain"))

[Document(id='9f5b89c2-d55a-48c3-af0b-980f214a42ac', metadata={'name': 'Shravan mukkawar', 'description': 'Main group', 'category': 'Web Development', 'tags': ['React']}, page_content='Group 2:\nName       : Shravan mukkawar\nDescription: Main group\nCategory   : Web Development\nTags       : React'), Document(id='e2d5ba16-421c-45fc-a573-4c79fd909b5d', metadata={'name': 'Prompt Engineering Mastery', 'description': 'A nerd space to discuss advanced prompting strategies for GPT, Claude, and open-source LLMs.', 'category': 'AI & ML', 'tags': ['PromptEngineering', 'LLM', 'ChatGPT', 'LangChain', 'Prompts']}, page_content='Group 69:\nName       : Prompt Engineering Mastery\nDescription: A nerd space to discuss advanced prompting strategies for GPT, Claude, and open-source LLMs.\nCategory   : AI & ML\nTags       : PromptEngineering,LLM,ChatGPT,LangChain,Prompts'), Document(id='585eef83-7e76-45db-ad15-70d076ff805c', metadata={'name': 'Metaverse Builders Collective', 'description': 'Unite 3D de

In [7]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a strict assistant. From the given context, identify the **five most relevant groups** for the user's question.
If there are groups which are not relevant remove it from the list.
Each group is described with fields like "Name       :", "Description:", etc.

Your task is to:
- Extract and return only the **group names**
- **Exactly as they appear** in the "Name       :" field
- Do not include any explanation or extra text

Return the group names as a numbered list.

Context:
{context}

Question:
{question}

Return format:
1. <Group Name 1>  
2. <Group Name 2>  
3. <Group Name 3>  
4. <Group Name 4>  
5. <Group Name 5>

"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [13]:
query = "Blockchain"
response = qa_chain.run(query)

print("Recommended Group:", response)


Recommended Group: Based on the given context, the five most relevant groups for the user's question are:

1. Group 2 - Shravan mukkawar (Main group)
2. Group 46 - Edge AI & On-Device Inference (Relevant to Blockchain)
3. Group 74 - Urban Sketchers Collective (Relevant to Art and Design)
4. Group 61 - Rustaceans United (Relevant to Programming Languages)
5. Group 20 - HealthTech & Digital Therapeutics (Relevant to Healthcare & Biotech)

Therefore, the list of relevant groups is:

1. Shravan mukkawar
2. Edge AI & On-Device Inference
3. Urban Sketchers Collective
4. Rustaceans United
5. HealthTech & Digital Therapeutics


In [16]:
!pip install pymilvus

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------------ --------------------------- 1.3/4.3 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 2.1/4.3 MB 5.9 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.3 MB 4.9 MB/s eta 0:00:01
   ------------------------------- -------- 3.4/4.3 MB 4.3 MB/s eta 0:00:01
   ------------------------------------ --- 3.9/4.3 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.0 MB 3.0 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11

In [20]:
!pip install python-dotenv